

```
# This is formatted as code
```

## Dataframe Basics

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [ ]:
# connect to database
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-09 09:22:04--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.75MB/s    in 0.5s    

2022-03-09 09:22:05 (1.75 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
# start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HW_GC").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [ ]:
# count rows
df.count()

4366916

In [ ]:
# print df schema
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

# Transform Dataset to fit tables in schema file

In [ ]:
from pyspark.sql.types import DateType

# create review df
review_df = df.select('review_id','customer_id','product_id','product_parent','review_date')
review_df = review_df.withColumn('review_date',review_df['review_date'].cast(DateType()))
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781| 2015-08-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853| 2015-08-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660| 2015-08-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263| 2015-08-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171| 2015-08-31|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457| 2015-08-31|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602| 2015-08-31|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055| 2015-08-31|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438| 2015-08-31|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213| 2015-08-31|
|R369CEXHXHC6NQ|   13409446|B00EYAFTCQ|     332158165| 2015-08-31|
|R171PPIJXFONVI|    9451727|B00I0QHY32|      49243908| 2015-08

In [ ]:
# verify datatype
review_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [ ]:
# create product df
product_df = df.select('product_id','product_title').drop_duplicates()
product_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [ ]:
# create customer df
from pyspark.sql.types import IntegerType
counts_df = df.groupBy("customer_id").count().orderBy("customer_id")
counts_df = counts_df.withColumn('count',counts_df['count'].cast(IntegerType()))
customers_df = counts_df.withColumnRenamed('count', 'customer_count')
customers_df.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

In [ ]:
# creating vine df
vine_df = df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

#Postgres Setup

In [ ]:
# configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cgjaxat2x40a.us-west-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "condren2022", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# write df to RDS
review_df.write.jdbc(url=jdbc_url, table='s_review_id_table', mode=mode, properties=config)

In [26]:
product_df.write.jdbc(url=jdbc_url, table='s_products', mode=mode, properties=config)

In [27]:
customers_df.write.jdbc(url=jdbc_url, table='s_customers', mode=mode, properties=config)

In [28]:
vine_df.write.jdbc(url=jdbc_url, table='s_vine_table', mode=mode, properties=config)